In [1]:
%reload_kedro

2021-04-05 15:44:44,675 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2021-04-05 15:44:44,781 - root - INFO - Registered hooks from 2 installed plugin(s): kedro-mlflow-0.7.0
2021-04-05 15:44:44,809 - root - INFO - ** Kedro project xflats
2021-04-05 15:44:44,810 - root - INFO - Defined global variable `context`, `session` and `catalog`
2021-04-05 15:44:44,834 - root - INFO - Registered line magic `run_viz`


In [2]:
X_train = catalog.load('hp_train')
X_test = catalog.load('hp_test')
X_valid = catalog.load('hp_valid')
y_train = catalog.load('hp_y_train')
y_test = catalog.load('hp_y_test')
y_valid = catalog.load('hp_y_valid')

2021-04-05 15:44:44,866 - kedro.io.data_catalog - INFO - Loading data from `hp_train` (PickleDataSet)...
2021-04-05 15:44:44,955 - kedro.io.data_catalog - INFO - Loading data from `hp_test` (PickleDataSet)...
2021-04-05 15:44:44,983 - kedro.io.data_catalog - INFO - Loading data from `hp_valid` (PickleDataSet)...
2021-04-05 15:44:45,020 - kedro.io.data_catalog - INFO - Loading data from `hp_y_train` (PickleDataSet)...
2021-04-05 15:44:45,046 - kedro.io.data_catalog - INFO - Loading data from `hp_y_test` (PickleDataSet)...
2021-04-05 15:44:45,054 - kedro.io.data_catalog - INFO - Loading data from `hp_y_valid` (PickleDataSet)...


In [3]:
from functools import partial
from typing import Any, Dict
from kedro.config import ConfigLoader
from hyperopt import fmin, hp, tpe
import lightgbm as lgb
import mlflow
from mlflow import log_metric, log_params
from mlflow.lightgbm import log_model
from mlflow.tracking import MlflowClient

from sklearn.metrics import mean_absolute_error, mean_squared_log_error
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

import scipy
import pandas as pd
import numpy as np

import logging
logger = logging.getLogger(__name__)



In [4]:
def _get_experiment() -> str:
    conf_paths = ["../conf/local", "../conf/base"]
    conf_loader = ConfigLoader(conf_paths=conf_paths)
    conf_mlflow = conf_loader.get("mlflow.yml")
    experiment_name = conf_mlflow\
        .get("experiment").get("name")
    #client = MlflowClient(tracking_uri=conf_mlflow.get("mlflow_tracking_uri"))
    client = MlflowClient("sqlite:///../data/mlflow.db")
    experiments = client.list_experiments()
    lista = list(filter(lambda x: x.name==experiment_name, experiments))
    return lista[0].experiment_id if  len(lista)>0 else 0

In [16]:
_get_experiment()

2021-04-05 16:19:31,897 - alembic.runtime.migration - INFO - Context impl SQLiteImpl.
2021-04-05 16:19:31,899 - alembic.runtime.migration - INFO - Will assume non-transactional DDL.


'1'

In [18]:
conf_paths = ["../conf/local", "../conf/base"]
conf_loader = ConfigLoader(conf_paths=conf_paths)
conf_mlflow = conf_loader.get("mlflow.yml")
experiment_name = conf_mlflow\
    .get("experiment").get("name")
conf_mlflow.get("mlflow_tracking_uri")

'sqlite:///data/mlflow.db'

In [21]:
def _get_tracking_uri() -> str:
    conf_paths = ["../conf/local", "../conf/base"]
    conf_loader = ConfigLoader(conf_paths=conf_paths)
    conf_mlflow = conf_loader.get("mlflow.yml")
    experiment_name = conf_mlflow\
        .get("experiment").get("name")
    return conf_mlflow.get("mlflow_tracking_uri")

In [22]:
_get_tracking_uri()

'sqlite:///data/mlflow.db'

**UWAGA** trzeba odpalić kod w glownym repo inaczej tworzy folder `mlrun` tam gdzie notebook

In [6]:
def _objective(
    params: Dict,
    X_train: np.ndarray,
    X_test: np.ndarray,
    y_train: np.ndarray,
    y_test: np.ndarray) -> float:
    
    experiment_id = _get_experiment()
    
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    mlflow.lightgbm.autolog(log_input_examples=False, 
                            log_model_signatures=False, 
                            log_models=True, 
                            disable=False, 
                            exclusive=False, 
                            disable_for_unsupported_versions=False, 
                            silent=False)
    
    with mlflow.start_run(experiment_id=experiment_id, nested=True):
        params['deterministic'] = True
        params['objective'] = "regression_l2"
        params['boosting'] = "gbdt"
        params['metric'] = ['l1','mape']
        params['seed'] = '666'
        
        #log_params(params)
        
        train_params = {
            'num_boost_round': 300,
            'verbose_eval': False,
            'early_stopping_rounds': 10,
        }
        
        train_data = lgb.Dataset(X_train, label=y_train, params={'verbose': -1})
        test_data = lgb.Dataset(X_test, label=y_test, params={'verbose': -1})
        
        #logger.info('Fitting model')

        model = lgb.train(
            params,
            train_data,
            valid_sets=[train_data, test_data],
            valid_names=['train', 'valid'],
            **train_params,
        )
                
        y_pred = model.predict(X_test)
        y_pred_train = model.predict(X_train)

        r2 = r2_score(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        mae = median_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        
        r2_tr = r2_score(y_train, y_pred_train)
        mape_tr = mean_absolute_percentage_error(y_train, y_pred_train)
        mae_tr = median_absolute_error(y_train, y_pred_train)
        mse_tr = mean_squared_error(y_train, y_pred_train)
        
                   
        #log_metric("mae_tst", round(mae/1000,1))
        #log_metric("mae_tra", round(mae_tr/1000,1))
        
        #log_metric("r2_tst", r2)
        #log_metric("r2_tra", r2_tr)

        #log_model(lgb_model=bst, 
        #          artifact_path="model",
        #          registered_model_name="LightgbmPricePredictor",
        #         )

        
        return mae
        

In [7]:
def hp_tuning(
    X_train: np.ndarray,
    X_test: np.ndarray,
    y_train: np.ndarray,
    y_test: np.ndarray,
    parameters: Dict,
    ):

    space = {
        "learning_rate": hp.uniform("learning_rate", 0.001, 0.5),
        "max_bin": hp.randint("max_bin", 100, 10000),
        "max_depth": hp.randint("max_depth", 2, 25),
        "min_data_in_leaf": hp.randint("min_data_in_leaf", 10, 150),
        "num_leaves": hp.randint("num_leaves", 10, 300),
        "lambda_l1": hp.uniform("lambda_l1", 0.0, 0.5),
        "lambda_l2": hp.uniform("lambda_l2", 0.0, 2.0),
        "bagging_fraction": hp.uniform("bagging_fraction", 0.3, 1.0), 
        "bagging_freq": hp.randint("bagging_freq", 1, 10),       
        "feature_fraction": hp.uniform("feature_fraction", 0.3, 1.0)
        }
    
    name = "LightgbmPricePredictor"
    tags = {"framework": "Lightgbm"}
    desc = "This price predition model is build for Warsaw flats price assessments."

    client = MlflowClient("sqlite:///../data/mlflow.db")
    
    try:
        client.create_registered_model(name, tags, desc)
    except:
        pass
    
    best = fmin(
        partial(_objective, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test), 
        space,
        algo=tpe.suggest,
        max_evals=parameters["hp_number_of_experiments"])
    return best



In [8]:
results = hp_tuning(
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    parameters={"hp_number_of_experiments": 2},
    )

2021-04-05 15:44:45,937 - alembic.runtime.migration - INFO - Context impl SQLiteImpl.
2021-04-05 15:44:45,937 - alembic.runtime.migration - INFO - Will assume non-transactional DDL.
  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]2021-04-05 15:44:45,987 - hyperopt.tpe - INFO - build_posterior_wrapper took 0.002698 seconds
2021-04-05 15:44:45,988 - hyperopt.tpe - INFO - TPE using 0 trials
2021-04-05 15:44:46,044 - alembic.runtime.migration - INFO - Context impl SQLiteImpl.
2021-04-05 15:44:46,045 - alembic.runtime.migration - INFO - Will assume non-transactional DDL.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.889608 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 880229                  
[LightGBM] [Info] Number of data points in the train set: 30000, number of used features: 1542
 50%|█████     | 1/2 [01:20<01:20, 80.43s/trial, best loss: 42566.78755875409]2021-04-05 15:46:06,428 - hype

In [27]:
results

{'bagging_fraction': 0.649225804927887,
 'bagging_freq': 4,
 'feature_fraction': 0.6729394484805863,
 'lambda_l1': 0.2234498923931597,
 'lambda_l2': 0.8964440633850108,
 'learning_rate': 0.30032452983063834,
 'max_bin': 7572,
 'max_depth': 4,
 'min_data_in_leaf': 110,
 'num_leaves': 259}

In [32]:
for k in results:
    if type(results[k]) == np.int64:
        results[k] = results[k].astype(np.int32)
    elif type(results[k]) == np.float64:
        results[k] = results[k].astype(np.float32)
    else:
        pass

results

{'bagging_fraction': 0.64922583,
 'bagging_freq': 4,
 'feature_fraction': 0.6729394,
 'lambda_l1': 0.22344989,
 'lambda_l2': 0.8964441,
 'learning_rate': 0.30032453,
 'max_bin': 7572,
 'max_depth': 4,
 'min_data_in_leaf': 110,
 'num_leaves': 259}

In [26]:
import json
json.dumps(results)

TypeError: Object of type int64 is not JSON serializable